# The demo file of unfoledVBA for grayscale images

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy.io as sio
%matplotlib inline
import time
import random
from Model_files.unfoldedVBA_vec import VBA_class
from Model_files.tools import *
from Model_files.model_vec import *
from Model_files.initializationdic import *
import cv2
import traceback

In [ ]:
torch.cuda.is_available()

## 1- Test saved models
### 1.1- Test conditions

In [ ]:
dataset         = 'Flickr30'
name_kernel     = 'Gaussian'
noise_std_min   = 0.01 
noise_std_max   = 0.01 
noise_std_range = [noise_std_min, noise_std_max] # one fixed noise std, gvar = 0.0001
device = 'GPU'

In [ ]:
im_size         = (256,256)
im_range        = [0,1]       # minimal and maximal pixel values
test_conditions = [name_kernel, noise_std_range, im_size, im_range]

# Path to the train/val/test set and to the folder with the saved model
path_trainset      = 'Datasets/Trainsets'
path_valset      = 'Datasets/Valsets'
path_testset       = 'Datasets/Testsets'
path_save          = os.path.join('Trainings',name_kernel,'saved_model')
paths              = [path_testset,path_trainset,path_valset,path_save]

print('---------------------------------------------------------------------------------------------------')
print('Testset: %s'%(dataset))
print('Blur   : %s'%(name_kernel))
print('---------------------------------------------------------------------------------------------------')

### 1.1.1- Change RGB image to graysclae image $\in$ [0,1]

In [ ]:
path_rgb = 'Datasets/Groundtruth/full/' + dataset
path_gray = 'Datasets/Groundtruth/full1/' + dataset # grayscale image saved here
# create the path if it does not exist 
if not os.path.exists(path_gray):
    os.makedirs(path_gray)

In [ ]:
for img_name in os.listdir(path_rgb):
    if img_name =='.ipynb_checkpoints': continue
    path_read = os.path.join(path_rgb,img_name)
    rgb_weights = [0.1140, 0.5870, 0.2989]
    image = cv2.imread(path_read)
    image = np.around(np.dot(image[...,:3], rgb_weights)).astype(int)
    nor_image = (image-image.min())/(image.max()-image.min()) # the normalization of the image
    path_save = os.path.join(path_gray,str(os.path.splitext(img_name)[0])+'.mat')
    sio.savemat(path_save, {'image': nor_image})    

### 1.2- Create blurred test images

In [ ]:
size_testset = len([n for n in os.listdir(os.path.join('Datasets/Groundtruth/full1',dataset))]) # number of test images

In [ ]:
print('The size of the test set is',size_testset)

In [ ]:
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Creating %d blurred test images from %s using %s blur kernel...'%(size_testset,dataset, name_kernel),flush=True)
create_testset(dataset,'Datasets/Groundtruth',path_testset,name_kernel,noise_std_range,im_size)
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

### 1.3- Load a saved model

In [ ]:
# choose the model learned by greedy approach or N-N training
saved_model = 'greedy_approach'
#saved_model = 'N-N training'

In [ ]:
if saved_model == 'greedy_approach':
    path_save_block = os.path.join('Trainings',name_kernel,'final_model_greedy_block7')
else:
    path_save_block = os.path.join('Trainings',name_kernel,'final_model_N_N_block7')
paths              = [path_testset,path_trainset,path_valset,path_save] 

In [ ]:
network              = VBA_class(test_conditions, paths, mode='test',device = device)
path_layers          = os.path.join(path_save_block,'trained_model_MinLossOnVal.pt')
path_post_processing = os.path.join(path_save_block,'trained_post-processing_MinLossOnVal.pt')
network.model.load_state_dict(torch.load(path_layers))
network.last_layer.load_state_dict(torch.load(path_post_processing))
if device == 'CPU': 
    network.to('cpu')  
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Loaded unfoldedVBA layers from %s.'%(path_layers))
print('Loaded the post-processing layer from %s.'%(path_post_processing))
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

### 1.4- Test a model

In [ ]:
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)
print('Testing model %s ...'%(path_save),flush=True)
time_start = time.time()
network.test(dataset)
time_per_image = (time.time()-time_start)/size_testset
print('Average time per image: %.2f seconds'%(time_per_image),flush=True)
print('--------------------------------------------------------------------------------------------------------------------------------',flush=True)

## 2- Train a model
### 2.1- Change RGB image to grayscale $\in$ [0,1] for training and validation set

In [ ]:
path_rgb = 'Datasets/BSD500_COCO1000_train_val/train'
path_gray = 'Datasets/BSD500_COCO1000_train_val/train1'
# create the path if it does not exist 
if not os.path.exists(path_gray):
    os.makedirs(path_gray)

# path_rgb = 'Datasets/BSD500_COCO1000_train_val/val'
# path_gray = 'Datasets/BSD500_COCO1000_train_val/val1'
# # create the path if it does not exist
# if not os.path.exists(path_gray):
#     os.makedirs(path_gray)


for img_name in os.listdir(path_rgb):
    if img_name != '.ipynb_checkpoints':
        path_read = os.path.join(path_rgb,img_name)
        rgb_weights = [0.1140, 0.5870, 0.2989]
        image = cv2.imread(path_read)
        #image = np.around(np.dot(image[...,:3], rgb_weights)).astype(int)
        image = np.dot(image[...,:3], rgb_weights)
        nor_image = (image-image.min())/(image.max()-image.min()) # the normalization of the image
        #print('true image is ',nor_image)
        path_save = os.path.join(path_gray,str(os.path.splitext(img_name)[0])+'.mat')
        sio.savemat(path_save, {'image': nor_image})  

In [ ]:
# path_rgb = 'Datasets/BSD500_COCO1000_train_val/train'
# path_gray = 'Datasets/BSD500_COCO1000_train_val/train1'

path_rgb = 'Datasets/BSD500_COCO1000_train_val/val'
path_gray = 'Datasets/BSD500_COCO1000_train_val/val1'

In [ ]:
size_trainset = len([n for n in os.listdir(os.path.join(path_gray)) if n != '.ipynb_checkpoints']) # number of test images

In [ ]:
print('The size of the train set is',size_trainset)

### 2.2- Create blurry noisy image for training and validation and save

In [ ]:
path_groundtruth_train = 'Datasets/BSD500_COCO1000_train_val/train1'
path_trainset = 'Datasets/Trainsets'
# create the path if it does not exist
if not os.path.exists(path_trainset):
    os.makedirs(path_trainset)
im_size         = (256,256)
create_trainset(path_groundtruth_train,path_trainset,noise_std_range,im_size)

path_groundtruth_val = 'Datasets/BSD500_COCO1000_train_val/val1'
path_valset = 'Datasets/Valsets'
# create the path if it does not exist
if not os.path.exists(path_valset):
    os.makedirs(path_valset)
im_size         = (256,256)
create_trainset(path_groundtruth_val,path_valset,noise_std_range,im_size)

In [ ]:
size_trainset = len([n for n in os.listdir(os.path.join(path_trainset))]) # number of test images
size_valset = len([n for n in os.listdir(os.path.join(path_valset))]) # number of test images

In [ ]:
print('The size of the train set is',size_trainset)
print('The size of the val set is',size_valset)

In [ ]:
data= sio.loadmat('Datasets/Trainsets/000000001072_blur8.mat')
image = data['image']
h = data['h']
plt.figure()
plt.imshow(h,cmap='gray')
plt.show()

### 2.3- Training parameters
##### Please refer to unfoldedVBA_vec.py to see all training parameters (learning rates, batch size, loss function, etc...)

In [ ]:
lr_first_layer     = 5e-3  #learning rate to train the first layer
lr_greedy          = 5e-3   #learning rate to train the following layers during greedy approach
lr_lpp             = 1e-3   #learning rate to train the post-processing layer
lr_N_N             = 5e-5  #learning rate to train all the layers together + lpp during greedy approach
mode               ='all_layers'    
nb_epochs          = [1,1,2,2] #number of epochs for training the first layer, the remaining layers, the post-processing layer, N-N training respectively
network            = VBA_class(test_conditions, paths, mode,
                         lr_first_layer     = lr_first_layer,
                         lr_greedy          = lr_greedy,
                         lr_lpp             = lr_lpp,
                         lr_N_N             = lr_N_N,
                         nb_epochs          = nb_epochs,
                         device             = device)

### 2.4- Train a model

In [ ]:
random.seed(a=1, version=2)

In [ ]:
init(network) # use the initial parameter for the first layer
if device == 'CPU': 
    network.to('cpu')

In [ ]:
#This line opens a log file
with open("bug_log.txt", "w") as log:

    try:
        network.train()
        print("There is no bug.", file = log)
    except Exception:
        traceback.print_exc(file=log) 